In [ ]:
# Import Core.
%run "../../Core/main.py"

In [ ]:
df = pd.read_csv(
    IMPORT_PATH + "orders.csv", 
    parse_dates=["CreatedAt"]
)

In [ ]:
print(df.shape)
print(df.dtypes)
df.head(2)

In [ ]:
df = bookings.groupby(["BookingID", "Route"]).agg(
    products=("ClientID", "nunique"),
    date_bought=("CreatedAt", "max"),
    cost=("TotalCost", "max"),
    currency=("CurrencyCode", "max"),
    departure=("DepartureTime", "max"),
    arrival=("DeliveryTime", "max")
).reset_index()

df.head(2)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Histogram
axes[0, 0].hist(df['cost'], bins=10, edgecolor='black')
axes[0, 0].set_title('Histogram of cost')
axes[0, 0].set_xlabel('cost')
axes[0, 0].set_ylabel('Frequency')

axes[0, 1].boxplot(df['cost'])
axes[0, 1].set_title('Box Plot of cost')
axes[0, 1].set_ylabel('cost')

sns.kdeplot(df['cost'], fill=True, ax=axes[1, 0])
axes[1, 0].set_title('Density Plot of cost')
axes[1, 0].set_xlabel('cost')
axes[1, 0].set_ylabel('Density')

fig.delaxes(axes[1, 1])

plt.tight_layout()
plt.show()

In [ ]:
def cohens_d(mean1, mean2, std1, std2, n1, n2):
    """Calculate Cohen's d effect size."""
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    return (mean1 - mean2) / pooled_std

def cohens_results(df):
    """Calculate Cohen's d for each pair of routes."""
    cohen_d_results = []

    for (idx1, row1), (idx2, row2) in combinations(df.iterrows(), 2):
        route1, route2 = row1['route'], row2['route']
        mean1, mean2 = row1['mean'], row2['mean']
        std1, std2 = row1['std'], row2['std']
        n1, n2 = row1['count'], row2['count']
        
        if n1 > 30 and n2 > 30:
            d = cohens_d(mean1, mean2, std1, std2, n1, n2)
            sufficient = "sufficient"
        else:
            d = np.nan
            sufficient = "insufficient"

        cohen_d_results.append({
            'route1': route1,
            'route2': route2,
            'cohens_d': d,
            'sample_size_status': sufficient
        })

    return pd.DataFrame(cohen_d_results)

def plot_cohens(df):
    """Plot Cohen's d values for route pairs."""
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 8))

    if 'cohens_d' not in df.columns:
        raise KeyError("The DataFrame must contain a 'cohens_d' column.")

    sns.barplot(data=df, x='cohens_d', y='route1', orient='h')

    plt.title("Cohen's d Values for Route Pairs", fontsize=16)
    plt.xlabel("Cohen's d", fontsize=14)
    plt.ylabel("Route Pairs", fontsize=14)

    plt.axvline(0.2, linestyle='--', color='orange', label='Small Effect')
    plt.axvline(0.5, linestyle='--', color='blue', label='Medium Effect')
    plt.axvline(0.8, linestyle='--', color='red', label='Large Effect')

    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
cohen_d_results_df = cohens_results(df)
plot_cohens(cohen_d_results_df)

In [ ]:
cohen_d_results_df

In [ ]:
def levenes_test(df):
    """Generate data samples based on mean and standard deviation."""
    for (idx1, row1), (idx2, row2) in combinations(df.iterrows(), 2):
        data1 = np.random.normal(row1['mean'], row1['std'], size=int(row1['count']))
        data2 = np.random.normal(row2['mean'], row2['std'], size=int(row2['count']))    
        stat1, p1 = shapiro(data1)
        stat2, p2 = shapiro(data2)

    stat, p = levene(data1, data2)
    print("Levene's test p-value:", p)

In [ ]:
levenes_test(df)

In [ ]:
def t_test(df):
    """Perform Welch's t-test between pairs of routes to compare their means."""
    df['count'] = df['count'].astype(int)

    t_test_results = []

    for (idx1, row1), (idx2, row2) in combinations(df.iterrows(), 2):
        route1, route2 = row1['route'], row2['route']
        mean1, mean2 = row1['mean'], row2['mean']
        std1, std2 = row1['std'], row2['std']
        n1, n2 = row1['count'], row2['count']

        data1 = np.random.normal(mean1, std1, size=n1)
        data2 = np.random.normal(mean2, std2, size=n2)

        t_stat, p_value = ttest_ind(data1, data2, equal_var=False)

        t_test_results.append({
            'route1': route1,
            'route2': route2,
            't_stat': t_stat,
            'p_value': p_value
        })

    df_t_test = pd.DataFrame(t_test_results)

    significant_results = df_t_test[df_t_test['p_value'] < 0.05]
    non_significant_results = df_t_test[df_t_test['p_value'] > 0.05]
    print("T-Test Results:")
    print(df_t_test)
    print("\nSignificant Results (p < 0.05):")
    print(non_significant_results)

In [ ]:
t_test(df)

In [ ]:
def anova_test(df):
    """Perform ANOVA to compare means across multiple routes and calculate the correlation between routes and cost."""
    from scipy import stats

    anova_results = stats.f_oneway(
        *[df[df['route'] == route]['cost'] for route in df['route'].unique()]
    )
    print("ANOVA test result:")
    print(f"F-statistic: {anova_results.statistic}, p-value: {anova_results.pvalue}")

    df['route_code'] = pd.factorize(df['route'])[0]  # Encode routes as numerical values
    correlation = df[['route_code', 'cost']].corr().iloc[0, 1]  # Correlation between route_code and cost
    print(f"Correlation between route and cost: {correlation}")

In [ ]:
anova_test(df)

In [ ]:
def get_season(month):
    """Return the season for a given month."""
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    return 'Fall'


def seasonality_anova_correlation(df, date_):
    """Perform ANOVA and correlation analysis based on seasonality and product."""
    from scipy import stats

    df['month'] = pd.to_datetime(df[date_]).dt.month
    df['season'] = df['month'].apply(get_season)
    df['departure_date'] = df['delivery'].dt.date
    df['delivery_hour'] = df['delivery'].dt.hour
    df['delivery_day'] = df['delivery'].dt.day
    df['booking_month'] = df['date_bought'].dt.month
    df['booking_day'] = df['date_bought'].dt.day
    df['is_weekend_delivery'] = df['delivery'].dt.weekday >= 5
    df['is_weekend_booking'] = df['date_bought'].dt.weekday >= 5

    anova_results = []
    for product in df['product'].unique():
        group_data = df[df['product'] == product]['cost'].dropna()
        if len(group_data) > 1:  # Ensure there's more than one data point
            anova_results.append(group_data)
        else:
            print(f"{product}: Not enough data for ANOVA.")

    if len(anova_results) > 1:
        anova_results = stats.f_oneway(*anova_results)
        print("\nANOVA test results for product:")
        print(f"F-statistic: {anova_results.statistic:.4f}, p-value: {anova_results.pvalue:.4f}")
    else:
        print("\nNot enough product groups for ANOVA.")

    seasonal_groups = [
        df[df['season'] == season]['cost'].dropna()
        for season in df['season'].unique() if len(df[df['season'] == season]['cost'].dropna()) > 1
    ]

    if len(seasonal_groups) > 1:
        anova_results_season = stats.f_oneway(*seasonal_groups)
        print("\nANOVA test result for seasonality:")
        print(f"F-statistic: {anova_results_season.statistic:.4f}, p-value: {anova_results_season.pvalue:.4f}")
    else:
        print("\nNot enough seasonal groups for ANOVA.")

    df['product_code'] = pd.factorize(df['product'])[0]
    df['season_code'] = pd.factorize(df['season'])[0]

    correlation = df[['product_code', 'cost']].corr().iloc[0, 1]
    print(f"\nCorrelation between product and cost: {correlation:.4f}")

    correlation_season = df[['season_code', 'cost']].corr().iloc[0, 1]
    print(f"Correlation between season and cost: {correlation_season:.4f}")

In [ ]:
seasonality_anova_correlation(df, 'delivery')

In [ ]:
def compute_confidence_interval(data):
    from scipy import stats
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard error of the mean
    ci = sem * stats.t.ppf((1 + 0.95) / 2., len(data) - 1)  # 95% CI
    return mean, (mean - ci, mean + ci)

In [ ]:
def plot_seasonality(ci_df):
    plt.figure(figsize=(12, 8))
    sns.set(style="whitegrid")

    sns.pointplot(
        x='season', 
        y='Mean', 
        hue='product', 
        data=ci_df, 
        dodge=True, 
        join=False, 
        ci=None,
        markers="o", 
        capsize=0.1
    )

    for i in range(ci_df.shape[0]):
        row = ci_df.iloc[i]
        plt.errorbar(
            x=row['season'], 
            y=row['Mean'], 
            yerr=[[row['Mean'] - row['Lower_CI']], [row['Upper_CI'] - row['Mean']]], 
            fmt='none', 
            capsize=5, 
            color='black'
        )

    plt.title('Mean cost by Product and Season with 95% Confidence Intervals', fontsize=16)
    plt.xlabel('Season', fontsize=14)
    plt.ylabel('Mean cost (per order)', fontsize=14)
    plt.legend(title='Product', title_fontsize='13', fontsize='11')

    plt.tight_layout()
    plt.title("Correlation Matrix for Training Data", color="white")

    plt.show()

In [ ]:
grouped = df.groupby(['product', 'season'])['cost']

ci_results = grouped.apply(lambda x: compute_confidence_interval(x.dropna()))

ci_df = pd.DataFrame(ci_results.tolist(), columns=['Mean', '95% CI'], index=ci_results.index)

ci_df[['Lower_CI', 'Upper_CI']] = pd.DataFrame(ci_df['95% CI'].tolist(), index=ci_df.index)

ci_df = ci_df.reset_index()

plot_seasonality(ci_df)

In [ ]:
def plot_confidence_interval(df):
    from scipy import stats
    df_routes_summary = df.groupby("route")["RPP"].agg(["mean", "count", "std"])

    df_routes_summary["SEM"] = df_routes_summary["std"] / np.sqrt(df_routes_summary["count"])

    confidence_level = 0.95
    z_value = stats.norm.ppf((1 + confidence_level) / 2)

    df_routes_summary["Lower_CI"] = df_routes_summary["mean"] - z_value * df_routes_summary["SEM"]
    df_routes_summary["Upper_CI"] = df_routes_summary["mean"] + z_value * df_routes_summary["SEM"]

    df_routes_summary = df_routes_summary.reset_index()

    plt.figure(figsize=(12, 8))
    sns.set(style="whitegrid")

    sns.pointplot(
        x="route", 
        y="mean", 
        data=df_routes_summary, 
        dodge=True, 
        join=False, 
        ci=None,  # Don"t calculate internal CIs; we already have them
        markers="o", 
        capsize=0.1
    )

    for i in range(df_routes_summary.shape[0]):
        row = df_routes_summary.iloc[i]
        plt.errorbar(
            x=row["route"], 
            y=row["mean"], 
            yerr=[[row["mean"] - row["Lower_CI"]], [row["Upper_CI"] - row["mean"]]], 
            fmt="none", 
            capsize=5, 
            color="black"
        )

    plt.title("Mean RPP by Route with 95% Confidence Intervals", fontsize=16)
    plt.xlabel("Route", fontsize=14, color="white")
    plt.ylabel("Mean RPP (Ticket Price per Passenger)", fontsize=14, color="white")

    plt.xticks(rotation=90)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_confidence_interval(df)

In [ ]:
def test_cost_effect(df, column):
    """Perform a t-test to check if the 'cost' is affected by a given column."""
    from scipy import stats

    cost_true = df[df[column] == True]['cost']
    cost_false = df[df[column] == False]['cost']

    t_stat, p_value = stats.ttest_ind(cost_true, cost_false, equal_var=False)
    
    return t_stat, p_value

t_stat_departure, p_value_departure = test_cost_effect(df, 'is_weekend_departure')
print(f"T-test for 'is_weekend_departure': t-statistic = {t_stat_departure}, p-value = {p_value_departure}")

t_stat_booking, p_value_booking = test_cost_effect(df, 'is_weekend_booking')
print(f"T-test for 'is_weekend_booking': t-statistic = {t_stat_booking}, p-value = {p_value_booking}")

In [ ]:
def non_parametric_test(df, column):
    """Perform a Mann-Whitney U test to check if 'cost' is affected by a given column. """
    from scipy.stats import mannwhitneyu

    cost_true = df[df[column] == True]['cost']
    cost_false = df[df[column] == False]['cost']

    if len(cost_true) == 0 or len(cost_false) == 0:
        print(f"Error: One of the groups in '{column}' has no data.")
        return None, None

    u_stat, p_value = mannwhitneyu(cost_true, cost_false, alternative='two-sided')
    
    return u_stat, p_value


u_stat_departure, p_value_departure = non_parametric_test(df, 'is_weekend_departure')
if u_stat_departure is not None:
    print(f"Mann-Whitney U test for 'is_weekend_departure': U-statistic = {u_stat_departure}, p-value = {p_value_departure}")

u_stat_booking, p_value_booking = non_parametric_test(df, 'is_weekend_booking')
if u_stat_booking is not None:
    print(f"Mann-Whitney U test for 'is_weekend_booking': U-statistic = {u_stat_booking}, p-value = {p_value_booking}")


In [ ]:
def calculate_rank_biserial_correlation(U, n1, n2):
    """Calculate the rank-biserial correlation as an effect size."""
    return (2 * U) / (n1 * n2) - 1

u_stat_delivery, p_value_delivery = non_parametric_test(df, 'is_weekend_delivery')
if u_stat_delivery is not None:
    print(f"Mann-Whitney U test for 'is_weekend_delivery': U-statistic = {u_stat_delivery}, p-value = {p_value_delivery}")

    n1_delivery = len(df[df['is_weekend_delivery'] == True])
    n2_delivery = len(df[df['is_weekend_delivery'] == False])
    effect_size_delivery = calculate_rank_biserial_correlation(u_stat_delivery, n1_delivery, n2_delivery)
    print(f"Rank-biserial correlation for 'is_weekend_delivery': {effect_size_delivery}")

u_stat_booking, p_value_booking = non_parametric_test(df, 'is_weekend_booking')
if u_stat_booking is not None:
    print(f"Mann-Whitney U test for 'is_weekend_booking': U-statistic = {u_stat_booking}, p-value = {p_value_booking}")

    n1_booking = len(df[df['is_weekend_booking'] == True])
    n2_booking = len(df[df['is_weekend_booking'] == False])
    effect_size_booking = calculate_rank_biserial_correlation(u_stat_booking, n1_booking, n2_booking)
    print(f"Rank-biserial correlation for 'is_weekend_booking': {effect_size_booking}")

plt.figure(figsize=(12, 6))
sns.boxplot(x='is_weekend_delivery', y='cost', data=df)
plt.title('cost by Weekend delivery')
plt.xlabel('Is Weekend delivery')
plt.ylabel('cost')
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='is_weekend_booking', y='cost', data=df)
plt.title('cost by Weekend Booking')
plt.xlabel('Is Weekend Booking')
plt.ylabel('cost')
plt.show()


In [ ]:
# ML model

In [ ]:
X = df.drop(columns=["cost"], axis=1)
y = df["cost"]

In [ ]:
X_train.var()

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

In [ ]:
def fill_with_distribution(series, distribution):
    """Fills missing values in the Series by sampling from existing values based on their distribution."""
    n_missing = series.isnull().sum()
    if n_missing == 0:
        return series

    valid_values = distribution.index.intersection(series.dropna().unique())
    valid_distribution = distribution[valid_values]
    
    if valid_distribution.sum() == 0:
        return series  # Return original series if there's nothing to sample from

    sampled_values = np.random.choice(valid_values, size=n_missing, p=valid_distribution / valid_distribution.sum())
    series_filled = series.copy()
    series_filled[series.isnull()] = sampled_values
    return series_filled

In [ ]:
def assign_time_of_day(df):
    """Calculates the daypart of the flight."""
    df["hour"] = df["departure"].dt.hour
    bins = [0, 6, 18, 24]
    labels = ["Night", "Morning", "Night"]
    df["time_of_day"] = pd.cut(df["hour"], bins=bins, labels=labels, right=False, include_lowest=True, ordered=False)
    df = df.drop("hour", axis=1)
    return df

In [ ]:
def create_time_features(df):
    """Adds time-related features derived from the "departure" date for USA, Norway, and UK."""
    holidays_data, vacation_periods = get_holidays_and_vacation_periods()
    
    holiday_dates = {country: [] for country in holidays_data.keys()}
    for country, years in holidays_data.items():
        for year in years.values():
            holiday_dates[country].extend(year.values())

    df["departure"] = pd.to_datetime(df["departure"])
    df["quarter"] = df["departure"].dt.quarter
    df["day_of_week"] = df["departure"].dt.dayofweek
    df["month"] = df["departure"].dt.month
    df["is_holiday_USA"] = df["departure"].isin(holiday_dates["USA"])
    df["is_holiday_Norway"] = df["departure"].isin(holiday_dates["Norway"])
    df["is_holiday_UK"] = df["departure"].isin(holiday_dates["UK"])
    df["is_vacation_USA"] = df["departure"].apply(lambda x: any(start <= x <= end for start, end in vacation_periods.get("USA", [])))
    df["is_vacation_Norway"] = df["departure"].apply(lambda x: any(start <= x <= end for start, end in vacation_periods.get("Norway", [])))
    df["is_vacation_UK"] = df["departure"].apply(lambda x: any(start <= x <= end for start, end in vacation_periods.get("UK", [])))
    df["is_weekend"] = df["day_of_week"].isin([5, 6])
    df = assign_time_of_day(df)
    df = assign_same_day_flight(df)
    return df

# Handle missing values

In [ ]:
missing_values_train = df.isnull().sum()
print(missing_values_train[missing_values_train > 0])

# Transformation

In [ ]:
X_train = create_time_features(X_train)
X_val = create_time_features(X_val)
X_test = create_time_features(X_test)

In [ ]:
encoder = OneHotEncoder(sparse=False)

X_train_encoded = encoder.fit_transform(X_train[["some_column"]])
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out())
X_val_encoded = encoder.transform(X_val[["some_column"]])
X_val_encoded = pd.DataFrame(X_val_encoded, columns=encoder.get_feature_names_out())
X_test_encoded = encoder.transform(X_test[["some_column"]])
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out())

X_train = X_train.join(X_train_encoded).drop(columns=["some_column"])
X_val = X_val.join(X_val_encoded).drop(columns=["some_column"])
X_test = X_test.join(X_test_encoded).drop(columns=["some_column"])

# Correlations

In [ ]:
numeric_columns = X_train.select_dtypes(include=[np.number])

correlation_matrix_train = numeric_columns.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix_train, annot=True, fmt=".2f", cmap='coolwarm', square=True, annot_kws={"color": "white"})

plt.title("Correlation Matrix for Training Data", color="white")
plt.xlabel("Features", color="white")
plt.ylabel("Features", color="white")

plt.xticks(color="white")
plt.yticks(color="white")

plt.show()

In [ ]:
scaler = MinMaxScaler()

X_train["days_until_delivery"] = scaler.fit_transform(X_train[["days_until_delivery"]])
X_val["days_until_delivery"] = scaler.transform(X_val[["days_until_delivery"]])
X_test["days_until_delivery"] = scaler.transform(X_test[["days_until_delivery"]])

In [ ]:
xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42)

param_grid = {
    "learning_rate": [0.01, 0.1],
    "n_estimators": [100, 200],
    "max_depth": [3, 6],
    "min_child_weight": [10, 15],
    "subsample": [0.8, 1.0],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                           scoring="neg_mean_squared_error", cv=10, verbose=1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = -grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best CV MSE: {best_score}")

best_model = XGBRegressor(**best_params, objective="reg:squarederror", random_state=42)
best_model.fit(X_train, y_train)

y_val_pred = best_model.predict(X_val)
val_mse = mean_squared_error(y_val, y_val_pred)
print(f"Validation MSE: {val_mse}")

y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print(f"Test MSE: {test_mse}")


In [ ]:
best_model.feature_importances_

In [ ]:
def manual_prediction(input_data):
    input_df = pd.DataFrame([input_data])
    input_df = pd.get_dummies(input_df, drop_first=True)   
    input_df = input_df.reindex(columns=X_train.columns, fill_value=0)  # Align columns    
    prediction = best_model.predict(input_df)
    return prediction[0]

# Example usage:
input_data = {
    'days_until_delivery': 1,
    'day_of_week': 5, 'month': 3, 'is_weekend': True,
    'same_day_delivery': False
}

suggested_price = manual_prediction(input_data)
print(f"Suggested cost price: {round(suggested_price)}")